In [1]:
!pip install pandas-gbq -U
!pip install parquet
!pip install pyarrow


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [69]:
import pandas as pd
import pandas_gbq
import requests
import json
from google.cloud import bigquery
import os

##URI is example only, change it to suit your needs

In [75]:
response = requests.get('https://www.censtatd.gov.hk/api/get.php?id=210-06507&lang=en&full_series=1').text
response_info = json.loads(response)
response = pd.json_normalize(response_info['dataSet'])
print(response)
print(response.dtypes)
print(response.columns)

    SEX SEXDesc freq  period          sv svDesc  figure sd_value
0         Total    Y    1985  MDNHRWKUDE  Hours      24         
1     F  Female    Y    1985  MDNHRWKUDE  Hours      24         
2     M    Male    Y    1985  MDNHRWKUDE  Hours      24         
3         Total    Q  198503  MDNHRWKUDE  Hours      24         
4     F  Female    Q  198503  MDNHRWKUDE  Hours      24         
..   ..     ...  ...     ...         ...    ...     ...      ...
568   F  Female    Q  202212  MDNHRWKUDE  Hours      20        r
569   M    Male    Q  202212  MDNHRWKUDE  Hours      24        r
570       Total    Q  202303  MDNHRWKUDE  Hours      20         
571   F  Female    Q  202303  MDNHRWKUDE  Hours      20         
572   M    Male    Q  202303  MDNHRWKUDE  Hours      20         

[573 rows x 8 columns]
SEX         object
SEXDesc     object
freq        object
period      object
sv          object
svDesc      object
figure       int64
sd_value    object
dtype: object
Index(['SEX', 'SEXDesc', 'freq

In [76]:
# OPTIONAL: converting object dtypes to strings
response['period'] = response['period'].astype(int)

Everything should be the same as CSV link to BigQuery

In [78]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials/hk-re-analytics-cfdf597ce9d5.json'
client = bigquery.Client()

project_id = 'hk-re-analytics'
dataset_id = 'hkgov_open_data'
table_id = 'median-hours-of-work'

dataset_table_id = f'{dataset_id}.{table_id}'
full_table_id = f'{project_id}.{dataset_id}.{table_id}'

pandas_gbq.to_gbq(response, dataset_table_id, project_id=project_id)

table = client.get_table(full_table_id)

print(
    f'Loaded {table.num_rows} rows and {len(table.schema)} columns to {full_table_id}'
)

Loaded 573 rows and 8 columns to hk-re-analytics.hkgov_open_data.median-hours-of-work
